### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [25]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

import operator

from scipy.stats import norm

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [26]:
hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    'x4': ["interval", -2, 8],
    'x5': ["interval", -2, 8],
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [27]:


BUDGETS = [1 + i * 52 for i in range(19)]

### Optimization

In [28]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size, x2, x3, x4, x5):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    # sum = 0
    # sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    # return sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    sum += x2 ** 2 - 10 * math.cos(2 * math.pi * x2)
    sum += x3 ** 2 - 10 * math.cos(2 * math.pi * x3)
    sum += x4 ** 2 - 10 * math.cos(2 * math.pi * x4)
    sum += x5 ** 2 - 10 * math.cos(2 * math.pi * x5)
    return len(hyperparameterspace) * 10 + sum


    ################# EGGHOLDER #################
    return -epochs * math.sin(math.sqrt(abs(epochs - (batch_size + 47)))) - (batch_size + 47) * math.sin(math.sqrt(abs((batch_size + 47 + 0.5 * epochs))))


def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1], coordinates[2], coordinates[3], coordinates[4], coordinates[5])


BUDGET = 400

N_ITERS = 10

ALT_1 = "{"
ALT_2 = "{"
ALT_3 = "{"

REF_PER_STEP = 1

for i in range(50):
    REF_PER_STEP = i + 1

    print("REF_PER_STEP:", REF_PER_STEP)

    #reset_seeds()
    summe = 0
    for i in range(N_ITERS):

        optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                    hyperparameterspace, BUDGET, 0, 0.75, 
                                                    init_points=10*len(hyperparameterspace), 
                                                    alternative=0, ref_per_step=REF_PER_STEP)
        points = optimizer.fit()

        points.sort(key=operator.attrgetter('value'))
        summe += points[0].get_value()

    ALT_1 += "(" + str(REF_PER_STEP) + "," + str(summe/N_ITERS) + ")"

    #reset_seeds()

    summe = 0
    for i in range(N_ITERS):

        optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                    hyperparameterspace, BUDGET, 0, 0.35, 
                                                    init_points=8*len(hyperparameterspace), 
                                                    alternative=1, ref_per_step=REF_PER_STEP)
        points = optimizer.fit()

        points.sort(key=operator.attrgetter('value'))
        summe += points[0].get_value()

    ALT_2 += "(" + str(REF_PER_STEP) + "," + str(summe/N_ITERS) + ")"

    #reset_seeds()

    summe = 0
    for i in range(N_ITERS):

        optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                    hyperparameterspace, BUDGET, 0, 0.6, 
                                                    init_points=7*len(hyperparameterspace), 
                                                    alternative=2, ref_per_step=REF_PER_STEP)
        points = optimizer.fit()

        points.sort(key=operator.attrgetter('value'))
        summe += points[0].get_value()

        # x_values = []
        # y_values = []
        # for i in range(len(points)):
        #     x_values.append(points[i].get_coordinates()[0])
        #     y_values.append(points[i].get_coordinates()[1])

        # fig = plt.figure()
        # ax = plt.axes()
        # surface = plt.scatter(x_values, y_values, c="red")

        # plt.plot([-512, 512],[512, 512])
        # plt.plot([-512, 512],[-512, -512])
        # plt.plot([-512, -512],[512, -512])
        # plt.plot([512, 512],[512, -512])

        # plt.gca().set_aspect("equal")

        #plt.savefig("./Alternative_3.pgf",bbox_inches='tight' )

        #plt.show()

    ALT_3 += "(" + str(REF_PER_STEP) + "," + str(summe/N_ITERS) + ")"


ALT_1 += "}"
ALT_2 += "}"
ALT_3 += "}"

print(ALT_1+"\n")
print(ALT_2+"\n")
print(ALT_3+"\n")
    



REF_PER_STEP: 1
REF_PER_STEP: 2
REF_PER_STEP: 3
REF_PER_STEP: 4
REF_PER_STEP: 5
REF_PER_STEP: 6
REF_PER_STEP: 7
REF_PER_STEP: 8
REF_PER_STEP: 9
REF_PER_STEP: 10
REF_PER_STEP: 11
REF_PER_STEP: 12
REF_PER_STEP: 13
REF_PER_STEP: 14
REF_PER_STEP: 15
REF_PER_STEP: 16
REF_PER_STEP: 17
REF_PER_STEP: 18
REF_PER_STEP: 19
REF_PER_STEP: 20
REF_PER_STEP: 21
REF_PER_STEP: 22
REF_PER_STEP: 23
REF_PER_STEP: 24
REF_PER_STEP: 25
REF_PER_STEP: 26
REF_PER_STEP: 27
REF_PER_STEP: 28
REF_PER_STEP: 29
REF_PER_STEP: 30
REF_PER_STEP: 31
REF_PER_STEP: 32
REF_PER_STEP: 33
REF_PER_STEP: 34
REF_PER_STEP: 35
REF_PER_STEP: 36
REF_PER_STEP: 37
REF_PER_STEP: 38
REF_PER_STEP: 39
REF_PER_STEP: 40
REF_PER_STEP: 41
REF_PER_STEP: 42
REF_PER_STEP: 43
REF_PER_STEP: 44
REF_PER_STEP: 45
REF_PER_STEP: 46
REF_PER_STEP: 47
REF_PER_STEP: 48
REF_PER_STEP: 49
REF_PER_STEP: 50
{(1,48.71293502446924)(2,50.302621752270944)(3,53.02356274109237)(4,47.49361376157556)(5,45.63645829771873)(6,43.4672909424926)(7,51.67495600450866)(8,46.61027